# **Import libraries**


In [1]:
# Import libraries for interactive widgets and display
import ipywidgets as widgets
from ipywidgets import VBox, Button, FileUpload, Output, Label
from IPython.display import display, HTML

# Import libraries for image handling
from PIL import Image
import io

# Import libraries for numerical operations
import numpy as np

# Import TensorFlow for model loading and predictions
import tensorflow as tf

# Import library to handle file operations in Google Colab
from google.colab import files

# **Upload The Model**
**Note: The model should have the name of "best_model.h5.keras"**

In [2]:
# Upload the model checkpoint
files.upload()

{}

# **Model Prediction Interface**

In [3]:
# Mean and std values calculated for the training data
mean = np.array([0.52074927, 0.42581322, 0.38057297])      # Mean for each channel (R, G, B)
std  = np.array([0.27928742, 0.25401181, 0.2549746 ])      # Standard deviation for each channel (R, G, B)

# Load the TensorFlow model
model = tf.keras.models.load_model('best_model.h5.keras')  # Adjust the path to your model

# Preprocess the image
def preprocess_image(image):
    img       = image.resize((200, 200))                  # Resize image to match model input
    img_array = np.array(img) / 255.0                     # Normalize to [0, 1]
    img_array = (img_array - mean) / std                  # Apply mean and std normalization
    img_array = np.expand_dims(img_array, axis=0)         # Add batch dimension
    return img_array

def on_upload_change(change):
    # Load image that has been uploaded
    fn  = io.BytesIO(btn_upload.data[-1])
    img = Image.open(fn)
    img.load()

    # Clear previous output
    out_pl.clear_output()

    # Display the uploaded image
    with out_pl:
        display(img)

    # Store the image for classification
    global uploaded_image
    uploaded_image = img

def on_click_classify(change=''):
    if uploaded_image is None:
        with out_pl:
            print("Please upload an image first.")
        return

    # Clear previous output
    out_pl.clear_output()

    # Display the image
    with out_pl:
        ratio = uploaded_image.size[0] / uploaded_image.size[1]
        c = uploaded_image.copy()
        c.thumbnail([ratio * 200, 200])
        display(c)

    # Preprocess the image
    img_array = preprocess_image(uploaded_image)

    # Make prediction
    predictions = model.predict(img_array)

    # Ensure predictions is 2D
    if len(predictions.shape) == 1:
        predictions = np.expand_dims(predictions, axis=0)

    # Get the probabilities
    probabilities = predictions.flatten()

    # Define class names
    class_names = ['Real', 'Fake']  # Adjust if your model's classes are different

    # Clear previous labels
    for label in labels:
        label.value = ""

    # Check number of probabilities and handle accordingly
    if len(probabilities) == 2:
        real_prob = probabilities[0]
        fake_prob = 1 - real_prob
        labels[0].value = f"Real (prob: {real_prob:.2f})"
        labels[1].value = f"Fake (prob: {fake_prob:.2f})"
    else:
        # Handle case where there are more or fewer than expected probabilities
        if len(probabilities) > 0:
            real_prob = probabilities[0]
            labels[0].value = f"Real (prob: {real_prob:.2f})"
            # Compute missing probability
            fake_prob = 1 - real_prob if len(probabilities) == 1 else probabilities[1]
            labels[1].value = f"Fake (prob: {fake_prob:.2f})"
        else:
            labels[0].value = "Real (prob: N/A)"
            labels[1].value = "Fake (prob: N/A)"

# Setting up widgets for file upload and classification
btn_upload = widgets.FileUpload(accept='image/*', multiple=False)
btn_upload.style.button_color = 'lightblue'
btn_upload.style.font_weight = 'bold'
btn_upload.observe(on_upload_change, names='data')

btn_run = widgets.Button(description="Classify", button_style='success')
btn_run.on_click(on_click_classify)
btn_run.style.font_weight = 'bold'

labels = [widgets.Label() for _ in range(2)]  # Only need labels for two classes

out_pl = widgets.Output()
out_pl.layout.border   = '8px solid black'
out_pl.layout.margin   = '10px'
out_pl.layout.width    = '200px'
out_pl.layout.height   = '200px'
out_pl.layout.overflow = 'auto'

intro_label = widgets.Label("Please upload a picture of an image face:")
intro_label.style = {'font_weight': 'bold', 'font_size': '18px', 'color': 'darkblue'}

# CSS styling for the VBox container
style = """
    <style>
        .widget-label {
            font-size: 16px;
            font-weight: bold;
        }
        .widget-button {
            background-color: #4CAF50;
            color: white;
            border: none;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
        .widget-button:hover {
            background-color: #45a049;
        }
    </style>
"""
display(HTML(style))

# Apply CSS classes
btn_upload.add_class('widget-button')
btn_run.add_class('widget-button')

# Initialize uploaded image variable
uploaded_image = None

# Arrange widgets in a vertical box layout
wgs = [intro_label, btn_upload, btn_run, out_pl] + labels
interface = widgets.VBox(wgs)

display(interface)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step
